In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import sys
import os
import glob
import time

sys.path.append("..")
from bert import optimization
from bert import modeling
from bert import tokenization

bert_paths = {
    "rbtl3": "/Users/zifei/Downloads/chinese-bert/chinese_rbtl3_L-3_H-1024_A-16",
    "rbt-ext": "/Users/zifei/Downloads/chinese-bert/chinese_roberta_wwm_ext_L-12_H-768_A-12",
}
bert_dir = bert_paths["rbtl3"]
tokenizer = tokenization.FullTokenizer(bert_dir + "/vocab.txt", do_lower_case=True)

In [2]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
for line in open("sents_all.tsv"):
    parts = line.rstrip("\n").split("\t")
    sid, pid, sent = parts
    print(parts)
    break

['1', '1', '白鹿今何在？']


In [4]:
bert_config = modeling.BertConfig.from_json_file(bert_dir + "/bert_config_rbtl3.json")
bert_config

In [5]:
def create_placeholders(batch_size=None, seq_length=16):
    features = {
        "input_ids": tf.placeholder(tf.int32, [batch_size, seq_length]),
        "input_mask": tf.placeholder(tf.int32, [batch_size, seq_length]),
    }
    return features

features = create_placeholders()
features

{'input_ids': <tf.Tensor 'Placeholder:0' shape=(?, 16) dtype=int32>,
 'input_mask': <tf.Tensor 'Placeholder_1:0' shape=(?, 16) dtype=int32>}

In [6]:
# CREATE BERT MODEL

model = modeling.BertModel(
  config=bert_config,
  is_training=False,
  input_ids=features["input_ids"],
  input_mask=features["input_mask"],
  token_type_ids=None,
  use_one_hot_embeddings=False)

pooled_output = model.get_pooled_output()
# TODO: add an output embedding layer when training the model.

# Are the output embeddings semantically meaningful?


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
pooled_output

<tf.Tensor 'bert/pooler/dense/Tanh:0' shape=(?, 1024) dtype=float32>

In [16]:
init_checkpoint = bert_dir + '/bert_model.ckpt'
use_tpu = False

tvars = tf.trainable_variables()
scaffold_fn = None
(assignment_map,
 initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
     tvars, init_checkpoint)

# HACK for keras LayerNorm
for k in list(assignment_map.keys()):
    if k.endswith("/gamma"):
        del assignment_map[k]
    if k.endswith("/beta"):
        assignment_map[k[:-4]] = k[:-4]
        del assignment_map[k]
        
if use_tpu:
  def tpu_scaffold():
    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
    return tf.train.Scaffold()
  scaffold_fn = tpu_scaffold
else:
  tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

tf.logging.info("**** Trainable Variables ****")
for var in tvars:
  init_string = ""
  if var.name in initialized_variable_names:
    init_string = ", *INIT_FROM_CKPT*"
  print("  name = %s, shape = %s%s" % (var.name, var.shape, init_string))


INFO:tensorflow:**** Trainable Variables ****
  name = bert/embeddings/word_embeddings:0, shape = (21128, 1024), *INIT_FROM_CKPT*
  name = bert/embeddings/token_type_embeddings:0, shape = (2, 1024), *INIT_FROM_CKPT*
  name = bert/embeddings/position_embeddings:0, shape = (512, 1024), *INIT_FROM_CKPT*
  name = bert/embeddings/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
  name = bert/embeddings/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (1024,), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (1024,), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/value/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
  name = bert/encoder/layer_0/attention/self/value/bias:0, sh

In [20]:
sess = tf.Session()

AttributeError: module 'tensorflow_core.compat.v1' has no attribute 'global_variable_initializer'

In [23]:
sess.run(tf.global_variables_initializer())

In [25]:
def tokenize(sents_batch, seq_len=16):
    batch_ids = []
    batch_masks = []
    for s in sents_batch:
        tokens = ["[CLS]"] + tokenizer.tokenize(s) + ["[SEP]"]
        ids = tokenizer.convert_tokens_to_ids(tokens)
        ids = ids[:seq_len]
        ids = ids + [0] * (seq_len - len(ids))
        masks = [int(i>0) for i in ids]
        batch_ids.append(ids)
        batch_masks.append(masks)
    return batch_ids, batch_masks

print(tokenize(["你好世界", "你好世界你好世界你好世界你好世界你好世界"]))

([[101, 872, 1962, 686, 4518, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 872, 1962, 686, 4518, 872, 1962, 686, 4518, 872, 1962, 686, 4518, 872, 1962, 686]], [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [26]:
test_ids, test_masks = tokenize(["你好世界", "你好世界你好世界你好世界你好世界你好世界"])

In [39]:
%%time

embs = sess.run(pooled_output, feed_dict={
    features["input_ids"]: test_ids * 256,
    features["input_mask"]: test_masks * 256,
})
print(embs[0])
print(embs[1])

[ 0.80590755 -0.04846638  0.41607088 ...  0.0404962  -0.9999986
 -0.6127925 ]
[-0.17146362  0.16745898  0.3858084  ...  0.3777602  -0.99995214
 -0.33941993]
CPU times: user 25.3 s, sys: 2.2 s, total: 27.5 s
Wall time: 8.47 s


In [51]:
# COMPUTE ALL SENTENCE ENCODINGS. They are 1024 dimensional.
import time
import gzip

batch_size = 128  # larger may be faster on TPU but not necessarily on CPU
seq_len = 16
batch = []

OUTPUT_DIR = '/Volumes/WDMPBlue/Files/chinese-bert/poem-data/'
def encode_and_write(batch, fout):
    sents = [b[1] for b in batch]
    sids = [b[0] for b in batch]
    ids, masks = tokenize(sents, seq_len=seq_len)
    embs = sess.run(pooled_output, feed_dict={
        features["input_ids"]: ids,
        features["input_mask"]: masks,
    })
    assert len(sids) == len(embs)
    for sid, emb in zip(sids, embs):
        # Need to write bytes
        # gzfout.write("{}\t{}\n".format(sid, ",".join([str(e) for e in emb])).encode("utf-8"))
        fout.write("{}\t{}\n".format(sid, ",".join([str(e) for e in emb])))

    
# with gzip.open(OUTPUT_DIR + 'sent_encodings_all.tsv.gz', 'w') as gzfout:  # bottleneck

with open(OUTPUT_DIR + 'sent_encodings_all.tsv', 'w') as fout:
    for line in open("sents_all.tsv"):
        parts = line.rstrip("\n").split("\t")
        sid, pid, sent = parts
        if int(sid) % 10000 == 1:
            print("processing sentence {}".format(sid))
        if len(batch) == batch_size:
            encode_and_write(batch, fout)
            batch = []
        else:
            batch.append((sid, sent))

    if batch:  # last batch
        encode_and_write(batch, fout)

processing sentence 1
processing sentence 10001
processing sentence 20001
processing sentence 30001
processing sentence 40001
processing sentence 50001
processing sentence 60001
processing sentence 70001
processing sentence 80001
processing sentence 90001
processing sentence 100001
processing sentence 110001
processing sentence 120001
processing sentence 130001
processing sentence 140001
processing sentence 150001
processing sentence 160001
processing sentence 170001
processing sentence 180001
processing sentence 190001
processing sentence 200001
processing sentence 210001
processing sentence 220001
processing sentence 230001
processing sentence 240001
processing sentence 250001
processing sentence 260001
processing sentence 270001
processing sentence 280001
processing sentence 290001
processing sentence 300001
processing sentence 310001
processing sentence 320001
processing sentence 330001
processing sentence 340001
processing sentence 350001
processing sentence 360001
processing sent

In [48]:
with gzip.open('/Volumes/WDMPBlue/Files/chinese-bert/poem-data/test.tsv.gz', 'wb') as fout:
    fout.write(b"Hello\n")
    #fout.write(u"Hello")
    #fout.write(b"Hello")

In [17]:
assignment_map

OrderedDict([('bert/embeddings/position_embeddings',
              'bert/embeddings/position_embeddings'),
             ('bert/embeddings/token_type_embeddings',
              'bert/embeddings/token_type_embeddings'),
             ('bert/embeddings/word_embeddings',
              'bert/embeddings/word_embeddings'),
             ('bert/encoder/layer_0/attention/output/dense/bias',
              'bert/encoder/layer_0/attention/output/dense/bias'),
             ('bert/encoder/layer_0/attention/output/dense/kernel',
              'bert/encoder/layer_0/attention/output/dense/kernel'),
             ('bert/encoder/layer_0/attention/self/key/bias',
              'bert/encoder/layer_0/attention/self/key/bias'),
             ('bert/encoder/layer_0/attention/self/key/kernel',
              'bert/encoder/layer_0/attention/self/key/kernel'),
             ('bert/encoder/layer_0/attention/self/query/bias',
              'bert/encoder/layer_0/attention/self/query/bias'),
             ('bert/encoder/la

In [18]:
tvars

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(21128, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/valu

In [19]:
tf.train.list_variables(init_checkpoint)

[('bert/embeddings/LayerNorm/beta', [1024]),
 ('bert/embeddings/LayerNorm/gamma', [1024]),
 ('bert/embeddings/position_embeddings', [512, 1024]),
 ('bert/embeddings/token_type_embeddings', [2, 1024]),
 ('bert/embeddings/word_embeddings', [21128, 1024]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/beta', [1024]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/gamma', [1024]),
 ('bert/encoder/layer_0/attention/output/dense/bias', [1024]),
 ('bert/encoder/layer_0/attention/output/dense/kernel', [1024, 1024]),
 ('bert/encoder/layer_0/attention/self/key/bias', [1024]),
 ('bert/encoder/layer_0/attention/self/key/kernel', [1024, 1024]),
 ('bert/encoder/layer_0/attention/self/query/bias', [1024]),
 ('bert/encoder/layer_0/attention/self/query/kernel', [1024, 1024]),
 ('bert/encoder/layer_0/attention/self/value/bias', [1024]),
 ('bert/encoder/layer_0/attention/self/value/kernel', [1024, 1024]),
 ('bert/encoder/layer_0/intermediate/dense/bias', [4096]),
 ('bert/encoder/layer_0/intermed

In [ ]:
# # BERT inference

# def input_fn_builder(features, seq_length, is_training, drop_remainder):
#   """Creates an `input_fn` closure to be passed to TPUEstimator."""
